In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest,chi2

In [2]:
df=pd.read_csv("news_dataset.csv")
df

,label,text
0,REAL,Payal has accused filmmaker Anurag Kashyap of ...
1,FAKE,A four-minute-long video of a woman criticisin...
2,FAKE,"Republic Poll, a fake Twitter account imitatin..."
3,REAL,"Delhi teen finds place on UN green list, turns..."
4,REAL,Delhi: A high-level meeting underway at reside...
...,...,...
3724,REAL,19:17 (IST) Sep 20\n\nThe second round of coun...
3725,REAL,19:17 (IST) Sep 20\n\nThe second round of coun...
3726,FAKE,The Bengaluru City Police’s official Twitter h...
3727,REAL,"Sep 20, 2020, 08:00AM IST\n\nSource: TOI.in\n\..."


In [3]:
df.groupby('label').text.count()

label
FAKE    1871
REAL    1850
Name: text, dtype: int64

In [4]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
word=stopwords.words('english')

word

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [6]:
df.duplicated(subset="text").sum()
df.drop_duplicates(subset="text",keep="first",inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2230 entries, 0 to 3727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   2230 non-null   object
 1   text    2229 non-null   object
dtypes: object(2)
memory usage: 52.3+ KB


In [7]:
df.isnull().sum()


label    0
text     1
dtype: int64

In [8]:
df.dropna(axis=0,how='any',inplace=True)


In [9]:
df['length']=df['text'].apply(len)
df.head()

,label,text,length
0,REAL,Payal has accused filmmaker Anurag Kashyap of ...,538
1,FAKE,A four-minute-long video of a woman criticisin...,3406
2,FAKE,"Republic Poll, a fake Twitter account imitatin...",3084
3,REAL,"Delhi teen finds place on UN green list, turns...",2383
4,REAL,Delhi: A high-level meeting underway at reside...,5572


In [10]:
df['length'].describe()

count     2229.000000
mean      2762.330193
std       1381.747658
min         67.000000
25%       1928.000000
50%       2553.000000
75%       3346.000000
max      15782.000000
Name: length, dtype: float64

In [11]:
df.groupby('label').describe()


length                                                            \
        count         mean          std    min     25%     50%      75%   
label                                                                     
FAKE   1851.0  2938.211777  1323.989361  186.0  2116.5  2687.0  3445.50   
REAL    378.0  1901.068783  1336.246767   67.0  1113.5  1727.5  2288.75   

                
           max  
label           
FAKE   14486.0  
REAL   15782.0

In [12]:
# fake=df[df['label']=='FAKE'][0:377]
# real=df[df['label']=='REAL'][0:377]
# new_df=pd.concat([fake,real])
# new_df

In [13]:
# new_df=new_df.sample(frac=1)
# new_df

In [15]:
new_df=df[df['length']>1000]
new_df


,label,text,length
1,FAKE,A four-minute-long video of a woman criticisin...,3406
2,FAKE,"Republic Poll, a fake Twitter account imitatin...",3084
3,REAL,"Delhi teen finds place on UN green list, turns...",2383
4,REAL,Delhi: A high-level meeting underway at reside...,5572
5,REAL,ROME: Novak Djokovic knows it isn't model beha...,2880
...,...,...,...
3717,FAKE,"News reports claiming that a mob of 60,000 had...",3389
3719,FAKE,A set of images is being shared on Facebook wi...,1804
3720,FAKE,Barely 48 hours ahead of voting in the assembl...,4158
3722,FAKE,A quote by an impostor Facebook page of Financ...,1854


In [16]:
new_df.groupby('label').describe()

length                                                              \
        count         mean          std     min      25%     50%      75%   
label                                                                       
FAKE   1806.0  2994.820044  1289.913072  1021.0  2165.25  2732.0  3469.00   
REAL    300.0  2225.980000  1310.971513  1001.0  1527.75  1943.5  2479.75   

                
           max  
label           
FAKE   14486.0  
REAL   15782.0

In [17]:
fake=new_df[new_df['label']=='FAKE'][0:298]
real=new_df[new_df['label']=='REAL'][0:298]
new1_df=pd.concat([fake,real])
new1_df

,label,text,length
1,FAKE,A four-minute-long video of a woman criticisin...,3406
2,FAKE,"Republic Poll, a fake Twitter account imitatin...",3084
6,FAKE,A viral image showing controversial Islamic pr...,2797
7,FAKE,Several photos are being shared with the misle...,2330
8,FAKE,The driver of the DMU train Arvind Kumar has N...,3710
...,...,...,...
3595,REAL,VISAKHAPATNAM: Migrant workers on their way to...,2561
3598,REAL,MI vs CSK:\n\nABU DHABI: The Indian Premier Le...,1393
3629,REAL,MALE: India has extended a $250 million loan t...,2691
3671,REAL,Superstar Priyanka Chopra Jonas has now been l...,1182


In [18]:
new1_df=new1_df.sample(frac=1)
new1_df

,label,text,length
589,REAL,Ranchi: Government’s effort to persuade the fo...,2042
334,FAKE,A nearly nine-year-old video showing Supreme C...,2995
31,FAKE,A photoshopped image of Bengali actress and mo...,1735
599,REAL,WASHINGTON: Enter Journey's Crossing Church in...,2291
392,FAKE,A man touches the casket containing the body o...,5421
...,...,...,...
2897,REAL,ABU DHABI: After Chennai Super Kings (CSK) def...,2011
3,REAL,"Delhi teen finds place on UN green list, turns...",2383
172,FAKE,A viral Whatsapp message - stating that the Lo...,2179
418,FAKE,"Gujarat MLA Jignesh Mevani, on Tuesday shared ...",1886


In [19]:
new1_df.groupby('label').describe()

length                                                              \
       count         mean          std     min      25%     50%      75%   
label                                                                      
FAKE   298.0  3059.332215  1251.175418  1063.0  2270.75  2822.5  3552.50   
REAL   298.0  2227.285235  1315.014410  1001.0  1523.25  1943.5  2503.25   

                
           max  
label           
FAKE   10648.0  
REAL   15782.0

In [20]:
import string
def text_process(mess):
    nopunc=[char for char in mess if char not in string.punctuation]
    nopunc=''.join(nopunc)
    return ''.join([word for word in nopunc.split() if word.lower() not in stopwords.words('english')])

In [21]:
new1_df['text'].apply(text_process)

589     RanchiGovernment’seffortpersuadeformerassistan...
334     nearlynineyearoldvideoshowingSupremeCourtlawye...
31      photoshoppedimageBengaliactressmodelNusratJaha...
599     WASHINGTONEnterJourneysCrossingChurchWashingto...
392     mantouchescasketcontainingbodyGauriLankeshRavi...
                              ...                        
2897    ABUDHABIChennaiSuperKingsCSKdefeatedMumbaiIndi...
3       DelhiteenfindsplaceUNgreenlistturnsglasssandNE...
172     viralWhatsappmessagestatingLondonSchoolEconomi...
418     GujaratMLAJigneshMevaniTuesdaysharedphotoshopp...
231     disturbingvideoshowingheinousassaultmobstrippa...
Name: text, Length: 596, dtype: object

In [22]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(new1_df['text'],new1_df['label'],test_size=0.3,random_state=0)
x_train
y_train

1912    REAL
2897    REAL
3595    REAL
665     REAL
1819    REAL
        ... 
83      REAL
151     FAKE
258     FAKE
607     FAKE
177     FAKE
Name: label, Length: 417, dtype: object

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
pipeline=Pipeline([('bow',CountVectorizer(analyzer=text_process)),('tfidf',TfidfTransformer()),('classifer',SVC())])
pipeline.fit(x_train,y_train)
predication=pipeline.predict(x_test)
predication

array(['FAKE', 'REAL', 'FAKE', 'REAL', 'REAL', 'REAL', 'FAKE', 'FAKE',
       'REAL', 'REAL', 'FAKE', 'REAL', 'FAKE', 'REAL', 'REAL', 'REAL',
       'FAKE', 'FAKE', 'REAL', 'FAKE', 'REAL', 'REAL', 'REAL', 'REAL',
       'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'FAKE',
       'REAL', 'REAL', 'REAL', 'FAKE', 'REAL', 'FAKE', 'FAKE', 'REAL',
       'REAL', 'FAKE', 'REAL', 'FAKE', 'REAL', 'REAL', 'FAKE', 'REAL',
       'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'FAKE', 'REAL', 'FAKE',
       'REAL', 'FAKE', 'REAL', 'REAL', 'FAKE', 'REAL', 'FAKE', 'REAL',
       'REAL', 'FAKE', 'REAL', 'FAKE', 'FAKE', 'REAL', 'REAL', 'FAKE',
       'REAL', 'REAL', 'REAL', 'REAL', 'FAKE', 'REAL', 'FAKE', 'REAL',
       'REAL', 'FAKE', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL', 'REAL',
       'REAL', 'REAL', 'FAKE', 'FAKE', 'REAL', 'REAL', 'FAKE', 'REAL',
       'FAKE', 'REAL', 'REAL', 'FAKE', 'REAL', 'FAKE', 'FAKE', 'FAKE',
       'FAKE', 'REAL', 'FAKE', 'REAL', 'REAL', 'FAKE', 'FAKE', 'REAL',
      

In [24]:
from sklearn import metrics
print('Accuracy:',metrics.accuracy_score(predication,y_test))

Accuracy: 0.8212290502793296


In [27]:
news=input("Enter news=")
news_df=pd.DataFrame({"original_news":[news]})
predict=pipeline.predict(news_df['original_news'])
predict

Enter news=Samantha Ruth Prabhu posted a BTS pic from Kushi sets. One of the many photos from the lot had a quote on 'solitude'.


array(['REAL'], dtype=object)